## ML algorithms HyperOpt Search
* Sangwon Baek
* March 10th, 2023

In [1]:
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import kstest, norm, mannwhitneyu, chi2_contingency, fisher_exact, ttest_ind, zscore

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score, ParameterGrid, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm 
import time 
from sklearn.inspection import permutation_importance
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score, auc
from statsmodels.stats import contingency_tables

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

In [2]:
#Read csv file using pandas
df2 = pd.read_csv('../Data/Preprocessed/CRF_DD_removed.csv', low_memory=False)
df2 = df2.drop(columns='Unnamed: 0')

na_cols = ['UD_HT','UD_DM','UD_CVD','UD_cancer','UD_other','SMT_fever','SMT_cough','SMT_sputum','SMT_dyspnea',
          'SMT_myalgia', 'SMT_sorethroat', 'SMT_mental', 'SMT_GI', 'steroid']
df2[na_cols] = df2[na_cols].fillna(value=0)

#Remove D-Dimer from the dataframe
ColumnNames = ['No', 'ID', 'age', 'sex', 'dx_date', 'hospitalized_date', 'UD_HT', 'UD_DM', 'UD_CVD', 'UD_cancer',
              'UD_other', 'SMT_fever', 'SMT_cough', 'SMT_sputum', 'SMT_dyspnea', 'SMT_myalgia',
              'SMT_sorethroat', 'SMT_mental', 'SMT_GI', 'TX_0', 'TX_1', 'TX_2', 'TX_3', 'TX_4', 
               'Smoking_0', 'Smoking_1', 'Smoking_2', 'Smoking_3', 'BT', 'SBP', 
               'DBP', 'PR', 'RR', 'SPO2', 'WBC', 'ANC', 'ALC', 'PLT', 'CRP', 'LDH', 
               'DD', 'PCR', 'steroid', 'Mild','Moderate','Severe']
df2 = df2[ColumnNames].drop(columns = 'DD')

#Separate DF into validation and developmentsets
DevelopmentSet = df2.loc[~df2['ID'].isin(['SCH', 'SCN' 'SMC', 'JNU'])]
ValidationSet = df2.loc[df2['ID'].isin(['SCH', 'SCN' 'SMC', 'JNU'])]

#Convert the train and test dataset into X and Y
drop_columns_original = ['hospitalized_date', 'Mild','Moderate','Severe', 'dx_date', 'No', 'ID', 'PCR', 'steroid', 'TX_0', 'TX_1', 'TX_2', 'TX_3', 'TX_4', 'Smoking_0', 'Smoking_1', 'Smoking_2', 'Smoking_3']
DevelopmentSet_X = DevelopmentSet.drop(columns=drop_columns_original).values
DevelopmentSet_Y = DevelopmentSet[['Severe']].values.ravel()
ValidationSet_X = ValidationSet.drop(columns=drop_columns_original).values
ValidationSet_Y = ValidationSet[['Severe']].values.ravel()

#Standardize the dataset through standardScaler - No need to separate the dataset into train/test because we use 5fold CV for testing purpose 
scaler = StandardScaler()
scaler.fit(DevelopmentSet_X)
DevelopmentSet_X = scaler.transform(DevelopmentSet_X)
scaler.fit(ValidationSet_X)
ValidationSet_X = scaler.transform(ValidationSet_X)

### Stratified K fold - Setting Parameters

In [3]:
#Parameter setting for GridSearchCV
rf = RandomForestClassifier()
mlr = LogisticRegression()
xgb = XGBClassifier()
gbm = GradientBoostingClassifier()
svm = SVC()

#Stratified 5 Fold CV
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=0)

### Bayesian Optimization: HyperOPT

In [10]:
xgb_space = {'learning_rate': hp.choice('learning_rate', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 
                                                 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]),
              'max_depth' : hp.choice('max_depth', [i for i in range (3,20)]),
              'gamma' : hp.choice('gamma', [i/20.0 for i in range(0,20)]),
              'min_child_weight' : hp.choice('min_child_weight', [3,4,5,6,7,8,9,10,11,12,13,14,15]),     
              'eta' : hp.choice("eta", [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.12, 0.15, 0.18, 0.2, 0.22, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8]), 
              'nthread' : hp.choice("nthread", [3,4,5,6,7,8,9,10]), 
              'colsample_bytree':hp.choice('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])}
# GBM              
gbm_space = {"max_depth" : hp.choice('max_depth', [i for i in range (3,20)]),
             "learning_rate" : hp.choice('learning_rate', [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 
                                                 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]),
             "n_estimators" : hp.choice('n_estimators', [i for i in range(50,300)])}

# MLR
mlr_space = { 'C' : hp.choice('C', [0.0001, 0.0005, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 
                     0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3, 
                     0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]),
              'max_iter': hp.choice('max_iter', [i*5 for i in range (10,60)]),
              'solver': hp.choice('solver', ['liblinear', 'lbfgs']),
              'penalty': hp.choice('penalty', ['l2','l1', 'None'])}

# Random Forest
rf_space = {'n_estimators': hp.choice('n_estimators', [i for i in range(50,400)]),
            'max_depth': hp.choice('max_depth', [i for i in range (2,30)]),
            'min_samples_split': hp.choice('min_samples_split', [i for i in range (2,15)]),
            'max_features':hp.choice('max_features', ['sqrt','log2']),
            'criterion': hp.choice('criterion', ['gini','entropy'])}

# Support Vector Machine
svm_space = {'C': hp.choice('C', [0.0001, 0.0005, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 
                     0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3, 
                     0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]),
            'kernel': hp.choice('kernel', ['rbf']),
            'gamma': hp.choice('gamma',['scale', 'auto']),
            'probability': hp.choice('probability', [True])}

In [8]:
def objective(estimator, params, X, y, cv, scoring):
    score = cross_val_score(estimator=estimator, 
                            X=X, 
                            y=y, 
                            cv=cv, 
                            scoring=scoring, 
                            n_jobs=-1).mean()
    # Loss is negative score
    loss = - score
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

def optimize(estimator, X, y, cv, space, scoring, max_evals=200):
    def wrapped_objective(params):
        return objective(estimator, params, X, y, cv, scoring)
    
    best_rf = fmin(fn=wrapped_objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=Trials())
    return best_rf

In [11]:
#RF
best_rf = optimize(estimator=rf, X=DevelopmentSet_X, y=DevelopmentSet_Y, cv=cv, space=rf_space, scoring='roc_auc')
print(best_rf)
print(space_eval(rf_space, best_rf))

100%|██████████| 200/200 [05:01<00:00,  1.51s/trial, best loss: -0.8964549617712546]
{'criterion': 0, 'max_depth': 24, 'max_features': 1, 'min_samples_split': 11, 'n_estimators': 149}
{'criterion': 'gini', 'max_depth': 26, 'max_features': 'log2', 'min_samples_split': 13, 'n_estimators': 199}


In [9]:
#MLR
best_mlr = optimize(estimator=mlr, X=DevelopmentSet_X, y=DevelopmentSet_Y, cv=cv, space=mlr_space, scoring='roc_auc')
print(best_mlr)
print(space_eval(mlr_space, best_mlr))

100%|██████████| 200/200 [00:12<00:00, 15.49trial/s, best loss: -0.8864817281162428]
{'C': 7, 'max_iter': 15, 'penalty': 2, 'solver': 1}
{'C': 0.006, 'max_iter': 125, 'penalty': 'None', 'solver': 'lbfgs'}


In [8]:
#XGB
best_xgb = optimize(estimator=xgb, X=DevelopmentSet_X, y=DevelopmentSet_Y, cv=cv, space=xgb_space, scoring='roc_auc')
print(best_xgb)
print(space_eval(xgb_space, best_xgb))

100%|██████████| 200/200 [04:12<00:00,  1.26s/trial, best loss: -0.8824584041946986]
{'colsample_bytree': 1, 'eta': 13, 'gamma': 15, 'learning_rate': 25, 'max_depth': 15, 'min_child_weight': 11, 'nthread': 3}
{'colsample_bytree': 0.4, 'eta': 0.2, 'gamma': 0.75, 'learning_rate': 0.45, 'max_depth': 18, 'min_child_weight': 14, 'nthread': 6}


In [9]:
#GBM
best_gbm = optimize(estimator=gbm, X=DevelopmentSet_X, y=DevelopmentSet_Y, cv=cv, space=gbm_space, scoring='roc_auc')
print(best_gbm)
print(space_eval(gbm_space, best_gbm))

100%|██████████| 200/200 [06:16<00:00,  1.88s/trial, best loss: -0.8957368245201058]
{'learning_rate': 12, 'max_depth': 7, 'n_estimators': 41}
{'learning_rate': 0.15, 'max_depth': 10, 'n_estimators': 91}


In [6]:
#SVM
best_svm = optimize(estimator=svm, X=DevelopmentSet_X, y=DevelopmentSet_Y, cv=cv, space=svm_space, scoring='roc_auc')
print(best_svm)
print(space_eval(svm_space, best_svm))

100%|██████████| 200/200 [02:39<00:00,  1.26trial/s, best loss: -0.865977330414002]
{'C': 34, 'gamma': 1, 'kernel': 0, 'probability': 0}
{'C': 0.8, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}


In [ ]:
gscv_xgb = GridSearchCV (estimator = xgb, param_grid = param_xgb, scoring ='accuracy', cv = cv, refit=True, n_jobs=-1, verbose=1)
gscv_lgb = GridSearchCV (estimator = lgb, param_grid = param_lgb, scoring ='accuracy', cv = cv, refit=True, n_jobs=-1, verbose=1)
gscv_gbm = GridSearchCV (estimator = gbm, param_grid = param_gbm, scoring ='accuracy', cv = cv, refit=True, n_jobs=-1, verbose=1)
gscv_mlr = GridSearchCV (estimator = mlr, param_grid = param_mlr, scoring ='accuracy', cv = cv, refit=True, n_jobs=-1, verbose=1)
gscv_rf = GridSearchCV (estimator = rf, param_grid = param_rf, scoring ='accuracy', cv = cv, refit=True, n_jobs=-1, verbose=1)
gscv_svm = GridSearchCV (estimator = svm, param_grid = param_svm, scoring ='accuracy', cv = cv, refit=True, n_jobs=-1, verbose=1)

In [ ]:
gscv_xgb.fit(DevelopmentSet_X, DevelopmentSet_Y)

Fitting 5 folds for each of 1300 candidates, totalling 6500 fits


In [7]:
print('XGB parameter: ', gscv_xgb.best_params_)
print('XGB prediction accuracy: {:.4f}'.format(gscv_xgb.best_score_))

XGB parameter:  {'eta': 0.05, 'max_depth': 5, 'min_child_weight': 7, 'nthread': 3}
XGB prediction accuracy: 0.8918


In [8]:
gscv_lgb.fit(DevelopmentSet_X, DevelopmentSet_Y)

Fitting 5 folds for each of 3969 candidates, totalling 19845 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3, 0.4,
                                           0.5],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100],
                         'num_leaves': [20, 30, 40, 50, 60, 70, 80, 90, 100]},
             scoring='accuracy', verbose=1)

In [9]:
print('LGB parameter: ', gscv_lgb.best_params_)
print('LGB prediction accuracy: {:.4f}'.format(gscv_lgb.best_score_))

LGB parameter:  {'learning_rate': 0.5, 'max_depth': 9, 'n_estimators': 100, 'num_leaves': 70}
LGB prediction accuracy: 0.8915


In [10]:
gscv_gbm.fit(DevelopmentSet_X, DevelopmentSet_Y)

Fitting 5 folds for each of 2900 candidates, totalling 14500 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.04, 0.07, 0.1, 0.12, 0.14,
                                           0.16, 0.18, 0.2, 0.22, 0.24, 0.26,
                                           0.28, 0.3, 0.35, 0.4, 0.45, 0.5,
                                           0.55, 0.6],
                         'max_depth': [3, 4, 5, 6, 7],
                         'n_estimators': [10, 15, 20, 25, 30, 35, 40, 45, 50,
                                          55, 60, 65, 70, 75, 80, 85, 90, 95,
                                          100, 110, 120, 130, 140, 150, 160,
                                          170, 180, 190, 200]},
             scoring='accuracy', verbose=1)

In [11]:
print('GBM parameter: ', gscv_gbm.best_params_)
print('GBM prediction accuracy: {:.4f}'.format(gscv_gbm.best_score_))

GBM parameter:  {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 35}
GBM prediction accuracy: 0.8906


In [12]:
gscv_mlr.fit(DevelopmentSet_X, DevelopmentSet_Y)

Fitting 5 folds for each of 1872 candidates, totalling 9360 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.002, 0.003, 0.004,
                               0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02,
                               0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                               0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, ...],
                         'max_iter': [100, 110, 120, 130, 140, 150, 160, 170,
                                      180, 190, 200, 210, 220, 230, 240, 250,
                                      270, 290, 300, 320, 340, 360, 380, 400],
                         'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
             scoring='accuracy', verbose=1)

In [13]:
print('MLR parameter: ', gscv_mlr.best_params_)
print('MLR prediction accuracy: {:.4f}'.format(gscv_mlr.best_score_))

MLR parameter:  {'C': 0.005, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
MLR prediction accuracy: 0.8827


In [14]:
gscv_rf.fit(DevelopmentSet_X, DevelopmentSet_Y)

Fitting 5 folds for each of 3960 candidates, totalling 19800 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 3, 4, 5, 6],
                         'n_estimators': [70, 75, 80, 85, 90, 95, 100, 105, 110,
                                          120, 125, 130, 135, 140, 150, 160,
                                          170, 180]},
             scoring='accuracy', verbose=1)

In [15]:
print('RF parameter: ', gscv_rf.best_params_)
print('RF prediction accuracy: {:.4f}'.format(gscv_rf.best_score_))

RF parameter:  {'criterion': 'entropy', 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 70}
RF prediction accuracy: 0.8927


In [16]:
gscv_svm.fit(DevelopmentSet_X, DevelopmentSet_Y)

Fitting 5 folds for each of 190 candidates, totalling 950 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                               0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3,
                               0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75,
                               0.8, 0.85, 0.9, 0.95, 1, ...],
                         'gamma': ['scale', 'auto'], 'kernel': ['rbf'],
                         'probability': [True]},
             scoring='accuracy', verbose=1)

In [17]:
print('SVM parameter: ', gscv_svm.best_params_)
print('SVM prediction accuracy: {:.4f}'.format(gscv_svm.best_score_))

SVM parameter:  {'C': 0.8, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
SVM prediction accuracy: 0.8865
